In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error


file_path = 'C:\\Users\\BAE J.H\\Desktop\\독거노인비율.xlsx'  
df = pd.read_excel(file_path)

df.columns = ['연도', '65세 이상 인구', '65세 이상 1인가구수', '독거노인비율']
df.columns = df.columns.str.strip()

df['65세 이상 인구'] = df['65세 이상 인구'].replace({',': ''}, regex=True)
df['65세 이상 1인가구수'] = df['65세 이상 1인가구수'].replace({',': ''}, regex=True)

df['65세 이상 인구'] = pd.to_numeric(df['65세 이상 인구'], errors='coerce')
df['65세 이상 1인가구수'] = pd.to_numeric(df['65세 이상 1인가구수'], errors='coerce')
df['독거노인비율'] = pd.to_numeric(df['독거노인비율'], errors='coerce')



years = df['연도'].values.reshape(-1, 1)  
elderly_population = df['65세 이상 인구'].values  
one_person_households = df['65세 이상 1인가구수'].values  
elderly_rate = df['독거노인비율'].values / 100  

# 선형 회귀 모델을 사용
model_rate = LinearRegression()
model_rate.fit(years, elderly_rate)
predicted_rate = model_rate.predict(np.arange(2025, 2081).reshape(-1, 1))

model_population = LinearRegression()
model_population.fit(years, one_person_households * elderly_rate)
predicted_population_rate = model_population.predict(np.arange(2025, 2081).reshape(-1, 1))

predicted_population = predicted_population_rate / predicted_rate

X_train, X_test, y_train_rate, y_test_rate = train_test_split(years, elderly_rate, test_size=0.3, random_state=42)
X_train_population, X_test_population, y_train_population, y_test_population = train_test_split(years, one_person_households * elderly_rate, test_size=0.3, random_state=42)

model_rate.fit(X_train, y_train_rate)
model_population.fit(X_train_population, y_train_population)

train_score_rate = model_rate.score(X_train, y_train_rate)
train_score_population = model_population.score(X_train_population, y_train_population)

test_score_rate = model_rate.score(X_test, y_test_rate)
test_score_population = model_population.score(X_test_population, y_test_population)

predicted_df = pd.DataFrame({
    'Year': np.arange(2025, 2081),
    'Predicted Elderly Population': predicted_population.astype(int),
    'Predicted Rate': predicted_rate
})
predicted_df.columns = ['연도', '독거노인 수', '독거노인 비율']

output_path = 'C:\\Users\\BAE J.H\\Desktop\\독고노인 인구 예측.csv'
predicted_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"훈련 세트에서 독거노인 비율 예측 모델 R²: {train_score_rate:.4f}")
print(f"테스트 세트에서 독거노인 비율 예측 모델 R²: {test_score_rate:.4f}")
print(f"훈련 세트에서 독거노인 수 예측 모델 R²: {train_score_population:.4f}")
print(f"테스트 세트에서 독거노인 수 예측 모델 R²: {test_score_population:.4f}")


예측된 데이터가 'C:\Users\BAE J.H\Desktop\독고노인 인구 예측.csv' cti경로에 저장되었습니다.
훈련 세트에서 독거노인 비율 예측 모델 R²: 0.9072
테스트 세트에서 독거노인 비율 예측 모델 R²: 0.9310
훈련 세트에서 독거노인 수 예측 모델 R²: 0.9043
테스트 세트에서 독거노인 수 예측 모델 R²: 0.8843
